In [1]:
import pandas as pd 

from ftfy import fix_text
import unicodedata as ud

import os 
import re 

import matplotlib.pyplot as plt 

In [2]:
#load data 
path = "/workspaces/ERP_Newsletter/data_processed/newsletter_items.csv"
df = pd.read_csv(path)

In [3]:
#inspect 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1668 entries, 0 to 1667
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1668 non-null   object
 1   newsletter_number  1668 non-null   int64 
 2   issue_date         1668 non-null   object
 3   theme              1668 non-null   object
 4   subtheme           114 non-null    object
 5   title              1668 non-null   object
 6   description        1347 non-null   object
 7   link               1616 non-null   object
dtypes: int64(1), object(7)
memory usage: 104.4+ KB


In [4]:
print(f"Total rows: {len(df)}")
print(f"Unique newsletter: {df['newsletter_number'].nunique()}")

Total rows: 1668
Unique newsletter: 87


# Clean Up Text

In [5]:
# 1) First pass - general cleaning
def clean_cell(s: str) -> str:
    if not isinstance(s, str):
        return s
    # Fix mojibake and common bad encodings
    s = fix_text(s)
    # Normalize Unicode (e.g., composed/compatibility forms)
    s = ud.normalize("NFKC", s)
    # Optional: collapse weird whitespace
    s = " ".join(s.split())
    return s.strip()

# 2) Apply to all text columns
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].map(clean_cell)

In [6]:
# 3) Second pass - fix stubborn mojibake
MOJIBAKE_MARKERS = ("‚Ä", "â€", "Â", "â€“", "â€”")

def demojibake(s: str) -> str:
    if not isinstance(s, str) or not s:
        return s

    candidates = []
    candidates.append(fix_text(s))

    try:
        candidates.append(fix_text(s.encode("latin1").decode("utf-8")))
    except Exception:
        pass

    candidates.append(fix_text(fix_text(s)))

    repl = (s
        .replace("‚Äì", "–")
        .replace("‚Äî", "—")
        .replace("‚Äô", "’")
        .replace("‚Äò", "‘")
        .replace("‚Äú", "“")
        .replace("‚Äù", "”")
        .replace("Â ", " ")
        .replace("Â", "")
        .replace("â€“", "–")
        .replace("â€”", "—")
        .replace("â€˜", "‘")
        .replace("â€™", "’")
        .replace("â€œ", "“")
        .replace("â€\x9d", "”")
        .replace("â€¦", "…")
    )
    candidates.append(fix_text(repl))

    def score(t: str) -> int:
        return sum(t.count(m) for m in MOJIBAKE_MARKERS)

    best = min(candidates, key=score)
    best = ud.normalize("NFKC", best).strip()
    best = re.sub(r"\s+", " ", best)
    return best

def needs_fix(s: str) -> bool:
    if not isinstance(s, str):
        return False
    return any(m in s for m in MOJIBAKE_MARKERS)

for col in df.columns:
    if df[col].dtype == object:
        mask = df[col].map(needs_fix)
        if mask.any():
            df.loc[mask, col] = df.loc[mask, col].map(demojibake)

In [7]:
# 3) Final "nuclear" pass: force-replace stubborn mojibake everywhere
REPL = {
    # Common triples
    "‚Äì": "–", "‚Äî": "—",
    "‚Äô": "’", "‚Äò": "‘",
    "‚Äú": "“", "‚Äù": "”",

    # The older/other path
    "â€“": "–", "â€”": "—",
    "â€˜": "‘", "â€™": "’",
    "â€œ": "“", "â€\x9d": "”", "â€ť": "”",
    "â€¢": "•", "â€¦": "…",

    # Stray non-breaking space artifacts
    "Â ": " ",
    "Â": "",
}

obj_cols = [c for c in df.columns if df[c].dtype == object]
for c in obj_cols:
    s = df[c].astype(str)
    for bad, good in REPL.items():
        s = s.str.replace(bad, good, regex=False)
    # Collapse weird spacing once more
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    df[c] = s

df.to_csv("newsletters_clean.csv", index=False)
print("Forced replacements applied. Saved -> newsletters_clean.csv")


Forced replacements applied. Saved -> newsletters_clean.csv


In [8]:
MARKERS = ("‚Ä", "â€", "Â", "â€“", "â€”")
leftovers = {}

for c in df.columns:
    if df[c].dtype == object:
        m = df[c].astype(str).str.contains("|".join(map(repr, MARKERS)), regex=True)
        leftovers[c] = int(m.sum())

print("Leftovers per column:", leftovers)

# Show a few offending rows to eyeball
bad_mask = False
for m in MARKERS:
    bad_mask = bad_mask | df.astype(str).apply(lambda col: col.str.contains(m, na=False)).any(axis=1)
bad_rows = df[bad_mask].head(5)
print(bad_rows.to_string(index=False))

Leftovers per column: {'id': 0, 'issue_date': 0, 'theme': 0, 'subtheme': 0, 'title': 0, 'description': 0, 'link': 0}
Empty DataFrame
Columns: [id, newsletter_number, issue_date, theme, subtheme, title, description, link]
Index: []


# Check for Missing Values 

In [9]:
missing_counts = df.isna().sum()

missing_summary = pd.DataFrame({
    'Missing Values': missing_counts, 
    'Percentage (%)': (missing_counts / len(df)) * 100
})

missing_summary 

,Missing Values,Percentage (%)
id,0,0.0
newsletter_number,0,0.0
issue_date,0,0.0
theme,0,0.0
subtheme,0,0.0
title,0,0.0
description,0,0.0
link,0,0.0


# Remove items where description or link are missing

In [10]:
# Remove rows where 'description' or 'link' is missing
df_cleaned = df.dropna(subset=['description', 'link'])

# (Optional) Check how many rows remain
print(f"Rows before: {len(df)}")
print(f"Rows after : {len(df_cleaned)}")

# If you want to overwrite df:
df = df_cleaned

Rows before: 1668
Rows after : 1668


# Check for Duplicates 

In [11]:
#All rows identical 
total_duplicates = df.duplicated().sum()
print(f"Total duplicate rows (all columns identical): {total_duplicates}")

Total duplicate rows (all columns identical): 0


In [12]:
# Check duplicates where both title and link are the same
title_link_dupes = df[df.duplicated(subset=["title", "link"], keep=False)]

print(f"Number of duplicate title+link pairs: {title_link_dupes.shape[0]}")
title_link_dupes.sort_values(by=["title"]).head(10)

Number of duplicate title+link pairs: 117


,id,newsletter_number,issue_date,theme,subtheme,title,description,link
1301,bf6c4fd6-a5bd-48ca-9249-b5b92849e038,70,4 April 2025,Updates from the programme,nan,A reminder that the ESRC Education Research Pr...,"AI in Education: From chalkboards to chatbots,...",https://uk.bettshow.com/speakers/dominik-lukes
1327,0ad9176d-2b5a-4306-a9f3-2b4ffdf96be6,71,11 April 2025,Updates from the programme,nan,A reminder that the ESRC Education Research Pr...,"AI in Education: From chalkboards to chatbots,...",https://uk.bettshow.com/speakers/dominik-lukes
1311,9b882352-ae98-4018-ac34-72592bf97d7a,71,11 April 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1283,1efb810d-cb19-4874-8a0f-3ea29ff03205,70,4 April 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1235,7e3c8c89-8550-453d-b10b-49d9d30dbb26,68,21 March 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1629,04d84abf-3e45-4d42-a249-232136441c38,86,3 October 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,8 October 5.30pm – 7pm (Online) The latest in ...,https://www.ucl.ac.uk/education-research-progr...
1612,e1117506-e2bf-4f11-bffe-d6bfe0217e95,85,26 September 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,8 October 5.30pm – 7pm (Online) The latest in ...,https://www.ucl.ac.uk/education-research-progr...
1583,bd18da73-7f98-4c7b-a039-66fe82a622af,84,19 September 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,8 October 5.30pm – 7pm The latest in the What ...,https://www.ucl.ac.uk/education-research-progr...
877,3a937c33-27b1-4d4e-8b51-1e834195238c,50,18 October 2024,Updates from the programme,nan,BERJ Call for Papers - Making teaching an attr...,This special issue of BERJ is guest edited by ...,https://bera-journals.onlinelibrary.wiley.com/...
780,5e06ebc5-9779-4c58-9367-ddc9d80d648c,45,5 September 2024,Updates from the programme,nan,BERJ Call for Papers - Making teaching an attr...,This special issue of BERJ is guest edited by ...,https://bera-journals.onlinelibrary.wiley.com/...


In [13]:
# Count duplicates based on title only
title_dupes = df[df.duplicated(subset=["title"], keep=False)]

print(f"Number of rows with duplicate titles: {title_dupes.shape[0]}")
title_dupes.sort_values(by="title").head(10)


Number of rows with duplicate titles: 188


,id,newsletter_number,issue_date,theme,subtheme,title,description,link
1021,e8391654-1847-4630-82bc-69e634dc3645,57,13 December 2024,Political environment and key organisations,nan,10 Downing St - Break Down Barriers to Opportu...,nan,nan
1029,91050b70-2333-49f1-9a43-147d776436dd,57,13 December 2024,Political environment and key organisations,nan,10 Downing St - Break Down Barriers to Opportu...,Milestone: Giving children the best start in l...,https://www.gov.uk/missions/opportunity?mptk=f...
1327,0ad9176d-2b5a-4306-a9f3-2b4ffdf96be6,71,11 April 2025,Updates from the programme,nan,A reminder that the ESRC Education Research Pr...,"AI in Education: From chalkboards to chatbots,...",https://uk.bettshow.com/speakers/dominik-lukes
1301,bf6c4fd6-a5bd-48ca-9249-b5b92849e038,70,4 April 2025,Updates from the programme,nan,A reminder that the ESRC Education Research Pr...,"AI in Education: From chalkboards to chatbots,...",https://uk.bettshow.com/speakers/dominik-lukes
1283,1efb810d-cb19-4874-8a0f-3ea29ff03205,70,4 April 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1235,7e3c8c89-8550-453d-b10b-49d9d30dbb26,68,21 March 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1311,9b882352-ae98-4018-ac34-72592bf97d7a,71,11 April 2025,Updates from the programme,nan,"AI in Education: From chalkboards to chatbots,...","Speakers: Dominik Lukes , Lead Business Techno...",https://uk.bettshow.com/speakers/dominik-lukes
1648,15f17205-bafc-43d8-ba3d-b4881956e48b,87,10 October 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,Catch up with the video of the latest in the W...,https://mediacentral.ucl.ac.uk/Play/126585
1629,04d84abf-3e45-4d42-a249-232136441c38,86,3 October 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,8 October 5.30pm – 7pm (Online) The latest in ...,https://www.ucl.ac.uk/education-research-progr...
1612,e1117506-e2bf-4f11-bffe-d6bfe0217e95,85,26 September 2025,Updates from the Programme,nan,Addressing key issues in teacher recruitment a...,8 October 5.30pm – 7pm (Online) The latest in ...,https://www.ucl.ac.uk/education-research-progr...


In [14]:
# Count duplicates based on link only
link_dupes = df[df.duplicated(subset=["link"], keep=False)]

print(f"Number of rows with duplicate links: {link_dupes.shape[0]}")
link_dupes.sort_values(by="link").head(10)

Number of rows with duplicate links: 308


,id,newsletter_number,issue_date,theme,subtheme,title,description,link
877,3a937c33-27b1-4d4e-8b51-1e834195238c,50,18 October 2024,Updates from the programme,nan,BERJ Call for Papers - Making teaching an attr...,This special issue of BERJ is guest edited by ...,https://bera-journals.onlinelibrary.wiley.com/...
780,5e06ebc5-9779-4c58-9367-ddc9d80d648c,45,5 September 2024,Updates from the programme,nan,BERJ Call for Papers - Making teaching an attr...,This special issue of BERJ is guest edited by ...,https://bera-journals.onlinelibrary.wiley.com/...
698,eac10c5e-f50a-41b4-8dfd-07b69a5327ed,41,28 June 2024,Political environment and key organisations,nan,Big Education - Next Generation Schools Confer...,"14 October 10am – 4pm, London",https://bigeducation.org/product/next-generati...
601,17268e59-d380-4e8f-a2e7-964543717f17,35,10 May 2024,What matters in education?,nan,Big Education conference - 'Next Generation Sc...,Hear from schools across the country who are w...,https://bigeducation.org/product/next-generati...
202,a368f2ad-99ab-44f5-b079-e2b6050e5223,12,20 October 2023,Programme news,nan,By Gemma Moss,"In the run-up to the next general election, ea...",https://blogs.ucl.ac.uk/ioe/2023/10/16/how-sho...
213,239c8e48-2ed1-460c-863c-23bf430dd73c,12,20 October 2023,Programme news,nan,"How should research, policy and practice inter...",By Gemma Moss In the run-up to the next genera...,https://blogs.ucl.ac.uk/ioe/2023/10/16/how-sho...
990,1109e116-b2cc-4842-be5b-5a59f03bf3d9,56,6 December 2024,Updates from the programme,Embedding children's participation rights in p...,Save the date! The project's second annual sym...,nan,https://childrens-participation.org/
47,1d1ad500-8555-4988-8d4a-51a5a3d18566,3,20 July 2023,PI Updates and Papers,nan,My corona*: listening to children in corona times,Resource shared via Sarah Chicken's website - ...,https://childrens-participation.org/
343,3c14f150-5b6b-4288-a95e-7975feb50844,21,19 January 2024,News from the projects,nan,Children's participation in school's network h...,Project website.,https://childrens-participation.org/
451,26c3b40b-ce5d-419f-99a5-e04114319a88,28,8 March 2024,Update from the ERP projects,nan,The team began visiting primary schools across...,"Last week, the team held their second network ...",https://childrens-participation.org/


# Identify themes and subthemes

In [15]:
#Unique counts of columns 
print("Unique titles:", df["title"].nunique())
print("Unique themes:", df["theme"].nunique())
print("Unique subthemes", df["subtheme"].nunique())
print("Unique links:", df["link"].nunique())

Unique titles: 1541
Unique themes: 68
Unique subthemes 42
Unique links: 1450


In [16]:
### Add placeholders for missing themes/subhtemes

# 1) Normalize empties/whitespace/"nan"/"none" to real NA
df_norm = df.copy()
for col in ["theme", "subtheme"]:
    df_norm[col] = (
        df_norm[col]
        .astype("string")
        .replace(r"^\s*$", pd.NA, regex=True)   # empty/whitespace → NA
        .replace({"nan": pd.NA, "NaN": pd.NA, "None": pd.NA, "none": pd.NA})
    )

# 2) Create a version that fills NA with placeholders so ALL cases are counted
df_filled = df_norm.fillna({"theme": "No theme", "subtheme": "No subtheme"})

# 3) Group and count every (theme, subtheme) combo, including placeholder cases
theme_subtheme_counts = (
    df_filled
    .groupby(["theme", "subtheme"], dropna=False)
    .size()
    .reset_index(name="count")
    .sort_values(by=["theme", "subtheme"])
)

# 4) Export to Excel (preferred to avoid encoding issues)
out_path = "/workspaces/ERP_Newsletter/data_processed/theme_subtheme_counts.xlsx"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
theme_subtheme_counts.to_excel(out_path, index=False)

print(f"✅ Exported {len(theme_subtheme_counts)} rows to {out_path}")

# ---- Optional sanity checks ----
missing_theme_only = df_norm[df_norm["theme"].isna() & df_norm["subtheme"].notna()].shape[0]
missing_subtheme_only = df_norm[df_norm["subtheme"].isna() & df_norm["theme"].notna()].shape[0]
missing_both = df_norm[df_norm["theme"].isna() & df_norm["subtheme"].isna()].shape[0]

print("Rows with subtheme but NO theme:", missing_theme_only)
print("Rows with theme but NO subtheme:", missing_subtheme_only)
print("Rows with NEITHER theme nor subtheme:", missing_both)
print("Sum of counts equals original rows:",
      theme_subtheme_counts['count'].sum() == len(df))

✅ Exported 113 rows to /workspaces/ERP_Newsletter/data_processed/theme_subtheme_counts.xlsx
Rows with subtheme but NO theme: 0
Rows with theme but NO subtheme: 1554
Rows with NEITHER theme nor subtheme: 0
Sum of counts equals original rows: True


# Check Themes and Articles 

In [17]:
# Filter articles under themes

check_themes = df[df["theme"] == "Research – Practice – Policy"].copy()

# View a few examples
display(check_themes.head(20))

,id,newsletter_number,issue_date,theme,subtheme,title,description,link
390,2fbf7b7b-334a-4d82-977c-d14a5bbf4778,25,16 February 2024,Research – Practice – Policy,nan,CAPE - Quid pro quo? Why academics meet with p...,"Patrick McAlary, CAPE coordinator, explores wh...",https://t.co/DbEx7Z1PPJ
391,7f2bc1b1-fcd3-4602-89d3-cdd1d39845de,25,16 February 2024,Research – Practice – Policy,nan,The SHAPE of research impact,British Academy report exploring research impa...,https://www.thebritishacademy.ac.uk/publicatio...
392,1775fa9d-d0de-4be6-9e61-f466f7131eb2,25,16 February 2024,Research – Practice – Policy,nan,NFER Event – Disadvantaged Policy webinar,Thursday 22 February 2024 – 11am Online,https://www.nfer.ac.uk/events/disadvantaged-po...
407,e684b418-8313-40f5-a40d-1fb8e68784bc,26,23 February 2024,Research – Practice – Policy,nan,Post from the Co-Production Collective - How t...,Through her lived experience of working with c...,https://www.coproductioncollective.co.uk/news/...
408,c87df581-63e5-4351-855f-95df1e646d31,26,23 February 2024,Research – Practice – Policy,nan,From the Public Policy Design blog - Joining-u...,nan,https://publicpolicydesign.blog.gov.uk/2024/02...
409,a8733168-3fb2-4eb4-b5d4-d15318b122f8,26,23 February 2024,Research – Practice – Policy,nan,"BERA event - Social theory, educational resear...",22 May 2024 2pm – 4pm (Free for BERA members),https://www.bera.ac.uk/event/social-theory-edu...
419,4ac3413a-708a-4ed7-b168-cb98a7278fc0,27,1 March 2024,Research – Practice – Policy,nan,DfE – Future Engagement events to facilitate c...,If you would like to stay up to date with thes...,mailto:research.engagement@education.gov.uk
420,e1a2426d-1feb-471a-a830-b11e0c8932dd,27,1 March 2024,Research – Practice – Policy,nan,Event – The impact of Impact on evidence-infor...,"7th March, 16:30 - 17:30 Online panel discussi...",https://my.chartered.college/event/the-impact-...
421,0ccbd798-4d22-48df-b6ad-8686665bda27,27,1 March 2024,Research – Practice – Policy,nan,Schools Week - Teacher-powered research: how w...,A new stream of EEF evaluations aims to give t...,https://schoolsweek.co.uk/teacher-powered-rese...
422,650870c3-efec-4bde-b0ef-add400c94f6d,27,1 March 2024,Research – Practice – Policy,nan,Nuffield - Provocations: what are the issues s...,Nuffield want to fund research that can addres...,https://www.nuffieldfoundation.org/news/opinio...


# Rename Themes

In [18]:
# ---------- 0) Drop rows where the entire theme is the unsubscribe text
UNSUB_THEME = (
    "You have indicated that you are happy to receive news and updates from the "
    "ESRC Education Research Programme. To unsubscribe, please email "
    "Elizabeth.hudson@ucl.ac.uk with the word UNSUBSCRIBE in the title of the email."
)
mask_unsub = df["theme"].astype(str).str.strip().eq(UNSUB_THEME)
dropped_rows = int(mask_unsub.sum())
df = df[~mask_unsub].copy()

# ---------- 1) Normalizers
def norm_theme(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("—", "-").replace("–", "-")     # normalize dashes
    s = s.replace("’", "'").replace("‘", "'").replace("“", '"').replace("”", '"')
    return s.lower()

def norm_key(s: str) -> str:
    """Strong normalizer for matching keys like subthemes:
       - lowercase
       - & -> and
       - remove punctuation (commas, periods, NBSP)
       - normalize dashes
       - collapse spaces
    """
    if not isinstance(s, str):
        return ""
    s = s.strip().lower()
    s = s.replace("—", " ").replace("–", " ").replace("-", " ")
    s = s.replace("&", " and ")
    s = s.replace("’", "'").replace("‘", "'").replace("“", '"').replace("”", '"')
    s = re.sub(r"[,\.\u00A0]", " ", s)            # remove commas, periods, NBSP
    s = re.sub(r"[^a-z0-9\s]", " ", s)            # drop other punctuation
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------- 2) Theme mapping list: (new_theme, current_theme)
pairs = [
    # project_updates
    ("project_updates", "Embedding children's participation rights in pedagogical practice in lower primary classrooms in Wales PI: Sarah Chicken"),
    ("project_updates", "Investigating the recruitment and retention of ethnic minority teachers PI: Stephen Gorard"),
    ("project_updates", "News from the Projects"),
    ("project_updates", "News from the projects"),
    ("project_updates", "PI Updates and Papers"),
    ("project_updates", "PI: David Lundie"),
    ("project_updates", "Programme news"),
    ("project_updates", "Programme Update"),
    ("project_updates", "Programme update"),
    ("project_updates", "Project news"),
    ("project_updates", "Rethinking teacher recruitment: New approaches to attracting prospective STEM teachers PI: Rob Klassen"),
    ("project_updates", "Sustainable school leadership: comparing approaches to the training, supply and retention of senior school leaders across the UK PI Toby Greany"),
    ("project_updates", "Toby Greany"),
    ("project_updates", "Towards equity focused approaches to EdTech: a socio-technical perspective PI: Professor Rebecca Eynon"),
    ("project_updates", "Towards equity focused approaches to EdTech: a socio-technical perspective PI: Rebecca Eynon"),
    ("project_updates", "Update from the ERP projects"),
    ("project_updates", "Update from the ESRC Education Research Programme"),
    ("project_updates", "Update from the projects"),
    ("project_updates", "Update from UKRI"),  # singular: stays project_updates
    ("project_updates", "Updates from David Lundie"),
    ("project_updates", "Updates from Steph Ainsworth"),
    ("project_updates", "Updates from the ERP projects"),
    ("project_updates", "Updates from the ESRC"),
    ("project_updates", "Updates from the Programme"),
    ("project_updates", "Updates from the programme"),
    ("project_updates", "Updates from the projects"),
    ("project_updates", "Decentring the 'resilient teacher': exploring interactions between individuals and their social ecologies PI: Steph Ainsworth"),
    ("project_updates", "Four Nations"),
    ("project_updates", "Four Nations Landscape"),
    ("project_updates", "Four Nations landscape"),
    ("project_updates", "Peer reviewed articles from the ERP projects"),
    ("project_updates", "Peer reviewed publications from the ERP projects"),
    ("project_updates", "Seminar series topics"),
    ("project_updates", "Seminar topics"),

    # digital_ed
    ("digital_ed", "EdTech"),

    # political_context_and_organisations
    ("political_context_and_organisations", "What are the politicians saying?"),
    ("political_context_and_organisations", "What Matters in Education?"),
    ("political_context_and_organisations", "What matters in education?"),
    ("political_context_and_organisations", "4 Nations"),
    ("political_context_and_organisations", "4 Nations & key organisations"),
    ("political_context_and_organisations", "Political environment and key organisations"),
    ("political_context_and_organisations", "Political landscape - the election"),
    ("political_context_and_organisations", "Political landscape & key organisations"),
    ("political_context_and_organisations", "Political landscape across Four Nations & key organisations"),
    ("political_context_and_organisations", "Research – Practice – Policy"),
    ("political_context_and_organisations", "Calls for evidence"),
    ("political_context_and_organisations", "DfE"),
    ("political_context_and_organisations", "Education, Policy & Practice"),
    ("political_context_and_organisations", "EEF"),
    ("political_context_and_organisations", "ESRC"),
    ("political_context_and_organisations", "Politics"),
    ("political_context_and_organisations", "Launch of ESRC survey on social science research skills"),
    ("political_context_and_organisations", "Updates from UKRI"),  # plural: goes here
    # events_opportunities_research
    ("events_opportunities_research", "Conferences"),
    ("events_opportunities_research", "Opportunities"),
    ("events_opportunities_research", "Opportunities for funding"),
    ("events_opportunities_research", "Opportunities to blog"),
    ("events_opportunities_research", "Other Reports"),
    ("events_opportunities_research", "Other Research"),
    ("events_opportunities_research", "Relevant Events"),
    ("events_opportunities_research", "Relevant Research"),
    ("events_opportunities_research", "Reports"),
    ("events_opportunities_research", "Research"),
    ("events_opportunities_research", "Events"),
    # teacher_rrd
    ("teacher_rrd", "Teacher recruitment, retention & development"),
]

# ---------- 3) Build lookup (normalized)
lookup = {norm_theme(curr): new for new, curr in pairs}

# ---------- 4) Apply theme mapping (keep original if not matched)
theme_norm = df["theme"].map(norm_theme)
df["new_theme"] = theme_norm.map(lookup).fillna(df["theme"])

# ---------- 5) Robust subtheme-based overrides
sub_norm = df["subtheme"].map(norm_key)

# Map any variant of "Teacher recruitment, retention & development"
target_rrd = "teacher recruitment retention and development"
df.loc[sub_norm.eq(target_rrd), "new_theme"] = "teacher_rrd"

# Map any variant of "Digital"
df.loc[sub_norm.eq("digital"), "new_theme"] = "digital_ed"

# ---------- 6) Export to Excel
summary = (
    df.assign(theme_norm=theme_norm, subtheme_norm=sub_norm)
      .groupby(["new_theme", "theme_norm"])
      .size()
      .reset_index(name="count")
      .sort_values(["new_theme", "count"], ascending=[True, False])
)

out_dir = "/workspaces/ERP_Newsletter/data_processed"
os.makedirs(out_dir, exist_ok=True)
summary_path = os.path.join(out_dir, "theme_mapping_summary.xlsx")

with pd.ExcelWriter(summary_path) as xw:
    df.to_excel(xw, sheet_name="data_with_new_theme", index=False)
    summary.to_excel(xw, sheet_name="mapping_summary", index=False)

print(f"✅ Dropped {dropped_rows} unsubscribe row(s).")
print("✅ new_theme mapped from theme + subtheme overrides.")
print("📄 Excel written to:", summary_path)

✅ Dropped 28 unsubscribe row(s).
✅ new_theme mapped from theme + subtheme overrides.
📄 Excel written to: /workspaces/ERP_Newsletter/data_processed/theme_mapping_summary.xlsx


In [19]:
# ---------- 7) View unique new_theme values and their counts
theme_counts = (
    df["new_theme"]
    .value_counts(dropna=False)
    .reset_index()
    .rename(columns={"index": "new_theme", "new_theme": "count"})
)

print("🧭 Unique new_theme values and their counts:")
print(theme_counts)

🧭 Unique new_theme values and their counts:
                                 count  count
0  political_context_and_organisations    736
1                      project_updates    435
2                          teacher_rrd    216
3                           digital_ed    210
4        events_opportunities_research     43


# Number of unique domain names 

In [17]:
from urllib.parse import urlparse

# Extract domain names from the 'link' column
df["domain"] = df["link"].apply(lambda x: urlparse(str(x)).netloc if pd.notna(x) else None)

# Count unique domains
unique_domains = df["domain"].nunique()

print(f"🌐 There are {unique_domains} unique domains in this dataset.")

# Optional: see the top 10 most common domains
domain_counts = df["domain"].value_counts().reset_index()
domain_counts.columns = ["domain", "count"]
print(domain_counts.head(40))


🌐 There are 396 unique domains in this dataset.
                                   domain  count
0                       schoolsweek.co.uk    169
1                           www.ucl.ac.uk     95
2                              www.gov.uk     71
3                                             69
4                     theconversation.com     36
5                        forms.office.com     35
6                     www.theguardian.com     35
7                          www.bera.ac.uk     33
8                              epi.org.uk     29
9                          www.nfer.ac.uk     28
10                   www.eventbrite.co.uk     27
11                           www.gov.scot     23
12                www.education-ni.gov.uk     21
13                            www.tes.com     21
14               committees.parliament.uk     20
15  bera-journals.onlinelibrary.wiley.com     19
16             www.belfasttelegraph.co.uk     18
17                           www.ukri.org     16
18             fftedu

# Remove items with irrelevant domain names

In [18]:
# Count words
df["title_word_count"] = df["title"].astype(str).apply(lambda x: len(x.split()))
df["description_word_count"] = df["description"].astype(str).apply(lambda x: len(x.split()))

# Summary statistics
word_length_summary = df[["title_word_count", "description_word_count"]].describe().T
print(word_length_summary)


                         count       mean        std  min  25%   50%   75%  \
title_word_count        1668.0  14.923261  11.269193  1.0  9.0  12.0  17.0   
description_word_count  1668.0  26.751799  28.662520  1.0  4.0  20.0  38.0   

                          max  
title_word_count        168.0  
description_word_count  239.0  
